# Paris DeepAir Project - Data Exploration

Unité de mesure pour tous les polluants : mg/m3

Polluants mesurés :
- **CO** : monoxyde de carbone
- **NO2** : dioxyde d'azote
- **NO**: monoxyde d'azote
- **NOX** : oxydes d'azote
- **O3** : ozone
- **PM 10** : particules
- **PM 2,5** : particules fines
- **SO2** : dioxyde de souffre

Métaux mesurés:
- **ETBEN**: ethylbenzene
- **m+pXYL**: m+p-xylene
- **oXYL**: o-xylene
- **TOL**: toluene

## Imports

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
!cd ../data/pollution && ls -la

total 32
drwxr-xr-x@ 25 llm  staff    800 Nov 25 00:14 .
drwxr-xr-x@  7 llm  staff    224 Nov 28 13:09 ..
-rw-r--r--@  1 llm  staff  12292 Nov 29 14:35 .DS_Store
drwxr-xr-x@  3 llm  staff     96 Nov 29 14:44 1_Merged
drwxr-xr-x@  3 llm  staff     96 Nov 29 14:44 2_Processed
drwxr-xr-x@  6 llm  staff    192 Nov 21 21:43 75001_U_Halles
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:45 75002_T_Opera
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:45 75004_T_Quai_Celestins
drwxr-xr-x@  8 llm  staff    256 Nov 21 21:43 75006_T_Bonap
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:45 75007_Obs_Eiffel_3e
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:45 75007_U_Allee_R
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:46 75008_T_Champs_Elysees
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:44 75009_T_Haussmann
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:46 75012_T_BP_Est
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:46 75012_T_Bd_Soult
drwxr-xr-x@  7 llm  staff    224 Nov 21 21:46 75012_U_Rue_BaL
drwxr-xr-x@  7 llm  staff   

## Useful functions

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
import pandas as pd

def turbo_plot(X):
    fig = plt.figure(constrained_layout=True,figsize=(15,120))
    subfigs = fig.subfigures(X.shape[1], 1,squeeze=False,hspace=20)
    for outerind, subfig in enumerate(subfigs.flat):
        subfig.suptitle(f'Subfig {X.columns[outerind]}')
        axs = subfig.subplots(1, 3)
        sns.histplot(data = X, x = X.columns[outerind], kde=True, ax = axs[0])
        sns.boxplot(data = X, x = X.columns[outerind], ax = axs[1])
        qqplot(X[X.columns[outerind]],line='s',ax=axs[2])
    return plt.show()

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
from statsmodels.graphics.mosaicplot import mosaic
import pandas as pd
import numpy as np

def mosaic_plot(df,X,y, ax=None):
    default_colors =plt.rcParams['axes.prop_cycle'].by_key()['color']
    cross = pd.crosstab(df[X],df[y])
    couples = cross.unstack().index
    props = lambda x: {'facecolor': default_colors[int(x[0])],'edgecolor':'w'}
    labelizer = lambda k: {(str(cpl[0]),str(cpl[1])) : f'{cpl[0]}-{cpl[1]}\n{round(cross.loc[cpl[1],cpl[0]]/cross.loc[:,cpl[0]].sum()*100,2)}%'  for cpl in couples}[k]
    mosaic(df, [y, X],properties=props,labelizer = labelizer, ax=ax)
    
def turbo_plot(df, X, y,classification):
    fig = plt.figure(constrained_layout=True,figsize=(15,round(10/3*df.shape[1])))
    subfigs = fig.subfigures(X.shape[1], 1,squeeze=False,hspace=20)

    for outerind, subfig in enumerate(subfigs.flat):
        #plotting numerical features
        if X[X.columns[outerind]].dtypes not in ['object','categorical','string'] and round(X[X.columns[outerind]].nunique()/df.shape[0]*100,2)>9:
            subfig.suptitle(f'Subfig {X.columns[outerind]}')
            axs = subfig.subplots(1, 4)
            sns.histplot(data = X, x = X.columns[outerind], kde=True, ax = axs[0])
            sns.boxplot(data = X, x = X.columns[outerind], ax = axs[1])
            qqplot(X[X.columns[outerind]],line='s',ax=axs[2])
            if classification: 
                sns.stripplot(data = X, x = y, y=X.columns[outerind], hue=y, ax = axs[3])
            else: 
                sns.scatterplot(data = X, x = X.columns[outerind], y=y, ax = axs[3])

        #plotting categorical features
        else:
            subfig.suptitle(f'Subfig {X.columns[outerind]}')
            axs = subfig.subplots(1, 4)
            sns.countplot(data = X, x = X.columns[outerind], ax = axs[0],order=X[X.columns[outerind]].value_counts().sort_values(ascending=False).index)
            sns.countplot(data = X, x = X.columns[outerind], hue=y, ax = axs[1],order=X[X.columns[outerind]].value_counts().sort_values(ascending=False).index)
            mosaic_plot(df,X.columns[outerind],df.survived.name,ax=axs[2])
            if classification: 
                sns.stripplot(data = X, x = y, y=X.columns[outerind], hue=y, ax = axs[3])
            else:
                sns.scatterplot(data = X, x = X.columns[outerind], y=y, ax = axs[3])
    return plt.show()

def quick_check(df, target:str, classification=True, to_drop=None):
    if target not in df.columns:
        raise ValueError('target not in df.columns')
    if not isinstance(target,str):
        raise TypeError('target must str')
    if to_drop:
        if all(x in df.columns for x in to_drop):
            raise ValueError('all elements in to_drop are not in df.columns')

        if not isintance(to_drop,list) and isintance(to_drop,str):
            to_drop=[to_drop]
        else:
            raise TypeError('to_drop type must be list of string')
    
    #Checking nan
    check= round(df.isna().sum()/df.shape[0]*100,2).sort_values(ascending=False)
    filtered = check[check>0]
    print(f'You have : {len(filtered)} features over {len(check)} ({round(check[check>0].shape[0]/check.shape[0],2)}% of whole df) that include np.nan')    

    #Features with nan
    print(f'\nHave a look at these features (% of nan): {", ".join([f"{i}: {str(v)}%" for i,v in filtered.items()])}')

    #Features to drop
    super_drop = check[check>15]
    print(f'\nYou might want to drop these features: {", ".join(super_drop.index)}')
    imputation = df[check[(check>0) & (check<15)].index].dtypes
    
    print('\n')
    print(df.info())

    print('\n')
    print("Let's have a look at all the features")
    X=df.drop(columns=(target if not to_drop else [target_name]+to_drop))
    y=df[target]
    turbo_plot(df, X,y,classification)

In [5]:
def quick_check2(df, target:str, classification=True, to_drop=None):
    if target not in df.columns:
        raise ValueError('target not in df.columns')
    if not isinstance(target,str):
        raise TypeError('target must str')
    if to_drop:
        if all(x in df.columns for x in to_drop):
            raise ValueError('all elements in to_drop are not in df.columns')

        if not isintance(to_drop,list) and isintance(to_drop,str):
            to_drop=[to_drop]
        else:
            raise TypeError('to_drop type must be list of string')
    
    #Checking nan
    check= round(df.isna().sum()/df.shape[0]*100,2).sort_values(ascending=False)
    filtered = check[check>0]
    print(f'You have : {len(filtered)} features over {len(check)} ({round(check[check>0].shape[0]/check.shape[0],2)}% of whole df) that include np.nan')    

    #Features with nan
    print(f'\nHave a look at these features (% of nan): {", ".join([f"{i}: {str(v)}%" for i,v in filtered.items()])}')

    #Features to drop
    super_drop = check[check>15]
    print(f'\nYou might want to drop these features: {", ".join(super_drop.index)}')
    imputation = df[check[(check>0) & (check<15)].index].dtypes
    
    print('\n')
    print(df.info())

## Per station

In [6]:
#LOCAL_DATA_PATH_pollution = os.path.join(os.path.expanduser('~'), "code", "TheLab75", "ParisDeepAirProject", "data", "pollution")
#LOCAL_DATA_PATH_pollution_merged = os.path.join(os.path.expanduser('~'), "code", "TheLab75", "ParisDeepAirProject", "data", "pollution", "1_Merged")
#LOCAL_DATA_PATH_pollution_processed = os.path.join(os.path.expanduser('~'), "code", "TheLab75", "ParisDeepAirProject", "data", "pollution", "2_Processed")

LOCAL_DATA_PATH_pollution = '../data/pollution'
LOCAL_DATA_PATH_pollution_merged = '../data/pollution/1_Merged'
LOCAL_DATA_PATH_pollution_processed = '../data/pollution/2_Processed'

### 75001_U_Halles

In [7]:
file_list =  !find ../data/Pollution/75001_U_Halles | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75001 - Halles'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75001 - Halles'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()

base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75001_U_Halles.csv")

In [8]:
PA75001 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75001_U_Halles.csv", index_col=0).copy()
PA75001 = PA75001.rename(columns={
    "index":"Date_time",
    "PA01H:CO":"CO",
    "PA01H:PM10":"PM10",
    "PA01H:PM25":"PM25",
    "PA01H:NO2":"NO2",
    "PA01H:NO":"NO",
    "PA01H:NOX":"NOX",
    "PA01H:O3":"O3",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75001 = PA75001.drop(columns=['CO', 'NO', 'NOX'])
PA75001['SO2'] = float('NaN')
PA75001 = PA75001[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75001

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2019/01/01 01:00:00+00,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
1,2019/01/01 02:00:00+00,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
2,2019/01/01 03:00:00+00,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
3,2019/01/01 04:00:00+00,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
4,2019/01/01 05:00:00+00,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
...,...,...,...,...,...,...,...,...
33907,2022/11/13 20:00:00+00,19.9,22.5,25.4,0.1,NaN,75001 - Halles,Urbain
33908,2022/11/13 21:00:00+00,21.4,23.6,28.5,0.1,NaN,75001 - Halles,Urbain
33909,2022/11/13 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain
33910,2022/11/13 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75001 - Halles,Urbain


In [9]:
PA75001.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75001.csv", index=False)

In [10]:
quick_check2(PA75001,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): SO2: 100.0%, PM10: 33.68%, NO2: 29.27%, PM25: 27.66%, O3: 22.37%

You might want to drop these features: SO2, PM10, NO2, PM25, O3


<class 'pandas.core.frame.DataFrame'>
Int64Index: 33912 entries, 0 to 33911
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     33912 non-null  object 
 1   PM25          24533 non-null  float64
 2   PM10          22492 non-null  float64
 3   NO2           23987 non-null  float64
 4   O3            26325 non-null  float64
 5   SO2           0 non-null      float64
 6   Station_name  33912 non-null  object 
 7   Station_type  33912 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.3+ MB
None


### 75002_T_Opera

In [11]:
file_list =  !find ../data/Pollution/75002_T_Opera | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75002 - Opera'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75002 - Opera'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75002_T_Opera.csv")

In [12]:
PA75002 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75002_T_Opera.csv", index_col=0).copy()
PA75002 = PA75002.rename(columns={
    "index":"Date_time",
    "OPERA:PM10":"PM10",
    "OPERA:NO2":"NO2",
    "OPERA:NO":"NO",
    "OPERA:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75002 = PA75002.drop(columns=['NO', 'NOX'])
PA75002['PM25'] = float('NaN')
PA75002['O3'] = float('NaN')
PA75002['SO2'] = float('NaN')
PA75002 = PA75002[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75002

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
1,2018/01/01 02:00:00+00,NaN,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
2,2018/01/01 03:00:00+00,NaN,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
3,2018/01/01 04:00:00+00,NaN,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
4,2018/01/01 05:00:00+00,NaN,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
...,...,...,...,...,...,...,...,...
42211,2022/11/14 20:00:00+00,NaN,32.5,39.0,NaN,NaN,75002 - Opera,Traffic
42212,2022/11/14 21:00:00+00,NaN,30.1,31.9,NaN,NaN,75002 - Opera,Traffic
42213,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75002 - Opera,Traffic
42214,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75002 - Opera,Traffic


In [13]:
PA75002.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75002.csv", index=False)

In [14]:
quick_check2(PA75002,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, O3: 100.0%, SO2: 100.0%, PM10: 13.48%, NO2: 2.82%

You might want to drop these features: PM25, O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42216 entries, 0 to 42215
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42216 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          36525 non-null  float64
 3   NO2           41025 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42216 non-null  object 
 7   Station_type  42216 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75004_T_Quai_Celestins

In [15]:
file_list =  !find ../data/Pollution/75004_T_Quai_Celestins | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75004 - Quai Celestins'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75004 - Quai Celestins'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75004_T_Quai_Celestins.csv")

In [16]:
PA75004 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75004_T_Quai_Celestins.csv", index_col=0).copy()
PA75004 = PA75004.rename(columns={
    "index":"Date_time",
    "CELES:NO2":"NO2",
    "CELES:NO":"NO",
    "CELES:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75004 = PA75004.drop(columns=['NO', 'NOX'])
PA75004['PM25'] = float('NaN')
PA75004['PM10'] = float('NaN')
PA75004['O3'] = float('NaN')
PA75004['SO2'] = float('NaN')
PA75004 = PA75004[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75004

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,21.0,NaN,NaN,75004 - Quai Celestins,Traffic
1,2018/01/01 02:00:00+00,NaN,NaN,16.0,NaN,NaN,75004 - Quai Celestins,Traffic
2,2018/01/01 03:00:00+00,NaN,NaN,16.0,NaN,NaN,75004 - Quai Celestins,Traffic
3,2018/01/01 04:00:00+00,NaN,NaN,15.0,NaN,NaN,75004 - Quai Celestins,Traffic
4,2018/01/01 05:00:00+00,NaN,NaN,12.0,NaN,NaN,75004 - Quai Celestins,Traffic
...,...,...,...,...,...,...,...,...
42619,2022/11/14 20:00:00+00,NaN,NaN,35.0,NaN,NaN,75004 - Quai Celestins,Traffic
42620,2022/11/14 21:00:00+00,NaN,NaN,25.6,NaN,NaN,75004 - Quai Celestins,Traffic
42621,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75004 - Quai Celestins,Traffic
42622,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75004 - Quai Celestins,Traffic


In [17]:
PA75004.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75004.csv", index=False)

In [18]:
quick_check2(PA75004,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, PM10: 100.0%, O3: 100.0%, SO2: 100.0%, NO2: 3.39%

You might want to drop these features: PM25, PM10, O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42624 entries, 0 to 42623
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42624 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          0 non-null      float64
 3   NO2           41181 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42624 non-null  object 
 7   Station_type  42624 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75006_T_Bonap

In [19]:
file_list =  !find ../data/Pollution/75006_T_Bonap | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75006 - Bonap'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75006 - Bonap'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75006_T_Bonap.csv")

In [20]:
PA75006 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75006_T_Bonap.csv", index_col=0).copy()
PA75006 = PA75006.rename(columns={
    "index":"Date_time",
    "BONAP:NO2":"NO2",
    "BONAP:NO":"NO",
    "BONAP:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75006 = PA75006.drop(columns=['NO', 'NOX'])
PA75006['PM25'] = float('NaN')
PA75006['PM10'] = float('NaN')
PA75006['O3'] = float('NaN')
PA75006['SO2'] = float('NaN')
PA75006 = PA75006[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75006

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,25.0,NaN,NaN,75006 - Bonap,Traffic
1,2018/01/01 02:00:00+00,NaN,NaN,21.0,NaN,NaN,75006 - Bonap,Traffic
2,2018/01/01 03:00:00+00,NaN,NaN,24.0,NaN,NaN,75006 - Bonap,Traffic
3,2018/01/01 04:00:00+00,NaN,NaN,23.0,NaN,NaN,75006 - Bonap,Traffic
4,2018/01/01 05:00:00+00,NaN,NaN,20.0,NaN,NaN,75006 - Bonap,Traffic
...,...,...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,NaN,NaN,35.2,NaN,NaN,75006 - Bonap,Traffic
42692,2022/11/14 21:00:00+00,NaN,NaN,28.6,NaN,NaN,75006 - Bonap,Traffic
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75006 - Bonap,Traffic
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75006 - Bonap,Traffic


In [21]:
PA75006.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75006.csv", index=False)

In [22]:
quick_check2(PA75006,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, PM10: 100.0%, O3: 100.0%, SO2: 100.0%, NO2: 9.96%

You might want to drop these features: PM25, PM10, O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          0 non-null      float64
 3   NO2           38443 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42696 non-null  object 
 7   Station_type  42696 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75007_Obs_Eiffel_3e

In [23]:
file_list =  !find ../data/Pollution/75007_Obs_Eiffel_3e | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75007 - Eiffel_3e'
base['Station_type'] = 'Observatoire'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75007 - Eiffel_3e'
    new_df['Station_type'] = 'Observatoire'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75007_Obs_Eiffel_3e.csv")

In [24]:
PA75007_1 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75007_Obs_Eiffel_3e.csv", index_col=0).copy()
PA75007_1 = PA75007_1.rename(columns={
    "index":"Date_time",
    "EIFF3:NO2":"NO2",
    "EIFF3:SO2":"SO2",
    "EIFF3:NO":"NO",
    "EIFF3:NOX":"NOX",
    "EIFF3:O3":"O3",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75007_1 = PA75007_1.drop(columns=['NO', 'NOX'])
PA75007_1['PM25'] = float('NaN')
PA75007_1['PM10'] = float('NaN')
PA75007_1 = PA75007_1[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75007_1

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,2.0,81.0,2.0,75007 - Eiffel_3e,Observatoire
1,2018/01/01 02:00:00+00,NaN,NaN,2.0,82.0,1.0,75007 - Eiffel_3e,Observatoire
2,2018/01/01 03:00:00+00,NaN,NaN,3.0,77.0,2.0,75007 - Eiffel_3e,Observatoire
3,2018/01/01 04:00:00+00,NaN,NaN,3.0,76.0,2.0,75007 - Eiffel_3e,Observatoire
4,2018/01/01 05:00:00+00,NaN,NaN,3.0,75.0,1.0,75007 - Eiffel_3e,Observatoire
...,...,...,...,...,...,...,...,...
42499,2022/11/14 20:00:00+00,NaN,NaN,8.6,43.2,NaN,75007 - Eiffel_3e,Observatoire
42500,2022/11/14 21:00:00+00,NaN,NaN,6.5,48.7,NaN,75007 - Eiffel_3e,Observatoire
42501,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Eiffel_3e,Observatoire
42502,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Eiffel_3e,Observatoire


In [25]:
PA75007_1.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75007_1.csv", index=False)

In [26]:
quick_check2(PA75007_1,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, PM10: 100.0%, SO2: 51.93%, O3: 19.44%, NO2: 7.35%

You might want to drop these features: PM25, PM10, SO2, O3


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42504 entries, 0 to 42503
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42504 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          0 non-null      float64
 3   NO2           39378 non-null  float64
 4   O3            34241 non-null  float64
 5   SO2           20430 non-null  float64
 6   Station_name  42504 non-null  object 
 7   Station_type  42504 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75007_U_Allee_R

In [27]:
file_list =  !find ../data/Pollution/75007_U_Allee_R | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75007 - Allee_R'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75007 - Allee_R'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75007_U_Allee_R.csv")

In [28]:
PA75007_2 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75007_U_Allee_R.csv", index_col=0).copy()
PA75007_2 = PA75007_2.rename(columns={
    "index":"Date_time",
    "PA07:NO2":"NO2",
    "PA07:NO":"NO",
    "PA07:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75007_2 = PA75007_2.drop(columns=['NO', 'NOX'])
PA75007_2['PM25'] = float('NaN')
PA75007_2['PM10'] = float('NaN')
PA75007_2['O3'] = float('NaN')
PA75007_2['SO2'] = float('NaN')
PA75007_2 = PA75007_2[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75007_2

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/02 01:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Allee_R,Urbain
1,2018/01/02 02:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Allee_R,Urbain
2,2018/01/02 03:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Allee_R,Urbain
3,2018/01/02 04:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Allee_R,Urbain
4,2018/01/02 05:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Allee_R,Urbain
...,...,...,...,...,...,...,...,...
42643,2022/11/13 20:00:00+00,NaN,NaN,24.2,NaN,NaN,75007 - Allee_R,Urbain
42644,2022/11/13 21:00:00+00,NaN,NaN,26.5,NaN,NaN,75007 - Allee_R,Urbain
42645,2022/11/13 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Allee_R,Urbain
42646,2022/11/13 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75007 - Allee_R,Urbain


In [29]:
PA75007_2.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75007_2.csv", index=False)

In [30]:
quick_check2(PA75007_2,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, PM10: 100.0%, O3: 100.0%, SO2: 100.0%, NO2: 3.46%

You might want to drop these features: PM25, PM10, O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42648 entries, 0 to 42647
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42648 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          0 non-null      float64
 3   NO2           41171 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42648 non-null  object 
 7   Station_type  42648 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75008_T_Champs_Elysees

In [31]:
file_list =  !find ../data/Pollution/75008_T_Champs_Elysees | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75008 - Champs Elysees'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75008 - Champs Elysees'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75008_T_Champs_Elysees.csv")

In [32]:
PA75008 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75008_T_Champs_Elysees.csv", index_col=0).copy()
PA75008 = PA75008.rename(columns={
    "index":"Date_time",
    "ELYS:PM10":"PM10",
    "ELYS:NO2":"NO2",
    "ELYS:NO":"NO",
    "ELYS:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75008 = PA75008.drop(columns=['NO', 'NOX'])
PA75008['PM25'] = float('NaN')
PA75008['O3'] = float('NaN')
PA75008['SO2'] = float('NaN')
PA75008 = PA75008[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75008

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,21.3,20.0,NaN,NaN,75008 - Champs Elysees,Traffic
1,2018/01/01 02:00:00+00,NaN,30.1,52.0,NaN,NaN,75008 - Champs Elysees,Traffic
2,2018/01/01 03:00:00+00,NaN,29.4,31.0,NaN,NaN,75008 - Champs Elysees,Traffic
3,2018/01/01 04:00:00+00,NaN,24.6,34.0,NaN,NaN,75008 - Champs Elysees,Traffic
4,2018/01/01 05:00:00+00,NaN,22.1,22.0,NaN,NaN,75008 - Champs Elysees,Traffic
...,...,...,...,...,...,...,...,...
42403,2022/11/14 20:00:00+00,NaN,25.1,33.7,NaN,NaN,75008 - Champs Elysees,Traffic
42404,2022/11/14 21:00:00+00,NaN,NaN,25.4,NaN,NaN,75008 - Champs Elysees,Traffic
42405,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75008 - Champs Elysees,Traffic
42406,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75008 - Champs Elysees,Traffic


In [33]:
PA75008.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75008.csv", index=False)

In [34]:
quick_check2(PA75008,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, O3: 100.0%, SO2: 100.0%, PM10: 17.21%, NO2: 5.86%

You might want to drop these features: PM25, O3, SO2, PM10


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42408 entries, 0 to 42407
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42408 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          35109 non-null  float64
 3   NO2           39921 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42408 non-null  object 
 7   Station_type  42408 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75009_T_Haussmann

In [35]:
file_list =  !find ../data/Pollution/75009_T_Haussmann | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75009 - Haussmann'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75009 - Haussmann'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75009_T_Haussmann.csv")

In [36]:
PA75009 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75009_T_Haussmann.csv", index_col=0).copy()
PA75009 = PA75009.rename(columns={
    "index":"Date_time",
    "HAUS:PM10":"PM10",
    "HAUS:PM25":"PM25",
    "HAUS:NO2":"NO2",
    "HAUS:NO":"NO",
    "HAUS:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75009 = PA75009.iloc[:, [0, 7, 1, 2, 3, 4, 5, 6]]
PA75009 = PA75009.drop(columns=['NO', 'NOX'])
PA75009['O3'] = float('NaN')
PA75009['SO2'] = float('NaN')
PA75009 = PA75009[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75009

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,19.3,53.0,NaN,NaN,75009 - Haussmann,Traffic
1,2018/01/01 02:00:00+00,NaN,20.0,44.0,NaN,NaN,75009 - Haussmann,Traffic
2,2018/01/01 03:00:00+00,NaN,24.3,46.0,NaN,NaN,75009 - Haussmann,Traffic
3,2018/01/01 04:00:00+00,NaN,21.4,40.0,NaN,NaN,75009 - Haussmann,Traffic
4,2018/01/01 05:00:00+00,NaN,21.8,34.0,NaN,NaN,75009 - Haussmann,Traffic
...,...,...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,13.2,24.4,32.8,NaN,NaN,75009 - Haussmann,Traffic
42692,2022/11/14 21:00:00+00,NaN,NaN,29.4,NaN,NaN,75009 - Haussmann,Traffic
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75009 - Haussmann,Traffic
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75009 - Haussmann,Traffic


In [37]:
PA75009.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75009.csv", index=False)

In [38]:
quick_check2(PA75009,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): O3: 100.0%, SO2: 100.0%, PM25: 82.34%, NO2: 1.93%, PM10: 1.76%

You might want to drop these features: O3, SO2, PM25


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   PM25          7542 non-null   float64
 2   PM10          41946 non-null  float64
 3   NO2           41870 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42696 non-null  object 
 7   Station_type  42696 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75012_T_Bd_Soult

In [39]:
file_list =  !find ../data/Pollution/75012_T_Bd_Soult | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75012 - Bd Soult'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75012 - Bd Soult'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_T_Bd_Soult.csv")

In [40]:
PA75012_1 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_T_Bd_Soult.csv", index_col=0).copy()
PA75012_1 = PA75012_1.rename(columns={
    "index":"Date_time",
    "SOULT:NO2":"NO2",
    "SOULT:NO":"NO",
    "SOULT:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75012_1 = PA75012_1.drop(columns=['NO', 'NOX'])
PA75012_1['PM25'] = float('NaN')
PA75012_1['PM10'] = float('NaN')
PA75012_1['O3'] = float('NaN')
PA75012_1['SO2'] = float('NaN')
PA75012_1 = PA75012_1[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75012_1

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,14.0,NaN,NaN,75012 - Bd Soult,Traffic
1,2018/01/01 02:00:00+00,NaN,NaN,14.0,NaN,NaN,75012 - Bd Soult,Traffic
2,2018/01/01 03:00:00+00,NaN,NaN,16.0,NaN,NaN,75012 - Bd Soult,Traffic
3,2018/01/01 04:00:00+00,NaN,NaN,20.0,NaN,NaN,75012 - Bd Soult,Traffic
4,2018/01/01 05:00:00+00,NaN,NaN,18.0,NaN,NaN,75012 - Bd Soult,Traffic
...,...,...,...,...,...,...,...,...
42523,2022/11/14 20:00:00+00,NaN,NaN,45.1,NaN,NaN,75012 - Bd Soult,Traffic
42524,2022/11/14 21:00:00+00,NaN,NaN,32.8,NaN,NaN,75012 - Bd Soult,Traffic
42525,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75012 - Bd Soult,Traffic
42526,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75012 - Bd Soult,Traffic


In [41]:
PA75012_1.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75012_1.csv", index=False)

In [42]:
quick_check2(PA75012_1,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, PM10: 100.0%, O3: 100.0%, SO2: 100.0%, NO2: 5.02%

You might want to drop these features: PM25, PM10, O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42528 entries, 0 to 42527
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42528 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          0 non-null      float64
 3   NO2           40393 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42528 non-null  object 
 7   Station_type  42528 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75012_T_BP_Est

In [43]:
file_list =  !find ../data/Pollution/75012_T_BP_Est | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75012 - BP Est'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75012 - BP Est'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_T_BP_Est.csv")

In [44]:
PA75012_2 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_T_BP_Est.csv", index_col=0).copy()
PA75012_2 = PA75012_2.rename(columns={
    "index":"Date_time",
    "BP_EST:PM10":"PM10",
    "BP_EST:PM25":"PM25",
    "BP_EST:NO2":"NO2",
    "BP_EST:NO":"NO",
    "BP_EST:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75012_2 = PA75012_2.drop(columns=['NO', 'NOX'])
PA75012_2['O3'] = float('NaN')
PA75012_2['SO2'] = float('NaN')
PA75012_2 = PA75012_2[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75012_2

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,8.9,18.1,13.0,NaN,NaN,75012 - BP Est,Traffic
1,2018/01/01 02:00:00+00,8.2,14.7,15.0,NaN,NaN,75012 - BP Est,Traffic
2,2018/01/01 03:00:00+00,6.1,11.6,20.0,NaN,NaN,75012 - BP Est,Traffic
3,2018/01/01 04:00:00+00,8.6,14.4,25.0,NaN,NaN,75012 - BP Est,Traffic
4,2018/01/01 05:00:00+00,10.4,18.2,25.0,NaN,NaN,75012 - BP Est,Traffic
...,...,...,...,...,...,...,...,...
42643,2022/11/14 20:00:00+00,21.8,31.5,52.7,NaN,NaN,75012 - BP Est,Traffic
42644,2022/11/14 21:00:00+00,NaN,NaN,46.9,NaN,NaN,75012 - BP Est,Traffic
42645,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75012 - BP Est,Traffic
42646,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75012 - BP Est,Traffic


In [45]:
PA75012_2.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75012_2.csv", index=False)

In [46]:
quick_check2(PA75012_2,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): O3: 100.0%, SO2: 100.0%, PM25: 2.98%, NO2: 2.44%, PM10: 2.21%

You might want to drop these features: O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42648 entries, 0 to 42647
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42648 non-null  object 
 1   PM25          41375 non-null  float64
 2   PM10          41704 non-null  float64
 3   NO2           41608 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42648 non-null  object 
 7   Station_type  42648 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75012_U_Rue_BaL

In [47]:
file_list =  !find ../data/Pollution/75012_U_Rue_BaL | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75012 - Rue BaL'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75012 - Rue BaL'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_U_Rue_BaL.csv")

In [48]:
PA75012_3 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75012_U_Rue_BaL.csv", index_col=0).copy()
PA75012_3 = PA75012_3.rename(columns={
    "index":"Date_time",
    "PA12:NO2":"NO2",
    "PA12:NO":"NO",
    "PA12:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75012_3 = PA75012_3.drop(columns=['NO', 'NOX'])
PA75012_3['PM25'] = float('NaN')
PA75012_3['PM10'] = float('NaN')
PA75012_3['O3'] = float('NaN')
PA75012_3['SO2'] = float('NaN')
PA75012_3 = PA75012_3[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75012_3

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,8.0,NaN,NaN,75012 - Rue BaL,Urbain
1,2018/01/01 02:00:00+00,NaN,NaN,10.0,NaN,NaN,75012 - Rue BaL,Urbain
2,2018/01/01 03:00:00+00,NaN,NaN,9.0,NaN,NaN,75012 - Rue BaL,Urbain
3,2018/01/01 04:00:00+00,NaN,NaN,9.0,NaN,NaN,75012 - Rue BaL,Urbain
4,2018/01/01 05:00:00+00,NaN,NaN,9.0,NaN,NaN,75012 - Rue BaL,Urbain
...,...,...,...,...,...,...,...,...
42667,2022/11/13 20:00:00+00,NaN,NaN,25.6,NaN,NaN,75012 - Rue BaL,Urbain
42668,2022/11/13 21:00:00+00,NaN,NaN,29.2,NaN,NaN,75012 - Rue BaL,Urbain
42669,2022/11/13 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75012 - Rue BaL,Urbain
42670,2022/11/13 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75012 - Rue BaL,Urbain


In [49]:
PA75012_3.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75012_3.csv", index=False)

In [50]:
quick_check2(PA75012_3,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, PM10: 100.0%, O3: 100.0%, SO2: 100.0%, NO2: 3.89%

You might want to drop these features: PM25, PM10, O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42672 entries, 0 to 42671
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42672 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          0 non-null      float64
 3   NO2           41012 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42672 non-null  object 
 7   Station_type  42672 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75013_U_Eastman

In [51]:
file_list =  !find ../data/Pollution/75013_U_Eastman | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75013 - Eastman'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75013 - Eastman'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75013_U_Eastman.csv")

In [52]:
PA75013 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75013_U_Eastman.csv", index_col=0).copy()
PA75013 = PA75013.rename(columns={
    "index":"Date_time",
    "PA13:NO2":"NO2",
    "PA13:NO":"NO",
    "PA13:NOX":"NOX",
    "PA13:O3":"O3",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75013 = PA75013.drop(columns=['NO', 'NOX'])
PA75013['PM25'] = float('NaN')
PA75013['PM10'] = float('NaN')
PA75013['SO2'] = float('NaN')
PA75013 = PA75013[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75013

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,9.0,73.0,NaN,75013 - Eastman,Urbain
1,2018/01/01 02:00:00+00,NaN,NaN,9.0,73.0,NaN,75013 - Eastman,Urbain
2,2018/01/01 03:00:00+00,NaN,NaN,11.0,70.0,NaN,75013 - Eastman,Urbain
3,2018/01/01 04:00:00+00,NaN,NaN,11.0,67.0,NaN,75013 - Eastman,Urbain
4,2018/01/01 05:00:00+00,NaN,NaN,10.0,66.0,NaN,75013 - Eastman,Urbain
...,...,...,...,...,...,...,...,...
42667,2022/11/13 20:00:00+00,NaN,NaN,25.1,1.9,NaN,75013 - Eastman,Urbain
42668,2022/11/13 21:00:00+00,NaN,NaN,27.3,1.7,NaN,75013 - Eastman,Urbain
42669,2022/11/13 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75013 - Eastman,Urbain
42670,2022/11/13 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75013 - Eastman,Urbain


In [53]:
PA75013.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75013.csv", index=False)

In [54]:
quick_check2(PA75013,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, PM10: 100.0%, SO2: 100.0%, NO2: 2.61%, O3: 1.42%

You might want to drop these features: PM25, PM10, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42672 entries, 0 to 42671
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42672 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          0 non-null      float64
 3   NO2           41557 non-null  float64
 4   O3            42064 non-null  float64
 5   SO2           0 non-null      float64
 6   Station_name  42672 non-null  object 
 7   Station_type  42672 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75014_T_Basch

In [55]:
file_list =  !find ../data/Pollution/75014_T_Basch | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75014 - Basch'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75014 - Basch'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75014_T_Basch.csv")

In [56]:
PA75014 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75014_T_Basch.csv", index_col=0).copy()
PA75014 = PA75014.rename(columns={
    "index":"Date_time",
    "BASCH:CO":"CO",
    "BASCH:PM10":"PM10",
    "BASCH:NO2":"NO2",
    "BASCH:NO":"NO",
    "BASCH:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75014 = PA75014.drop(columns=['CO', 'NO', 'NOX'])
PA75014['PM25'] = float('NaN')
PA75014['O3'] = float('NaN')
PA75014['SO2'] = float('NaN')
PA75014 = PA75014[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75014

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,19.8,23.0,NaN,NaN,75014 - Basch,Traffic
1,2018/01/01 02:00:00+00,NaN,14.0,23.0,NaN,NaN,75014 - Basch,Traffic
2,2018/01/01 03:00:00+00,NaN,14.9,30.0,NaN,NaN,75014 - Basch,Traffic
3,2018/01/01 04:00:00+00,NaN,19.9,27.0,NaN,NaN,75014 - Basch,Traffic
4,2018/01/01 05:00:00+00,NaN,21.4,25.0,NaN,NaN,75014 - Basch,Traffic
...,...,...,...,...,...,...,...,...
28963,2022/11/14 20:00:00+00,NaN,32.4,41.8,NaN,NaN,75014 - Basch,Traffic
28964,2022/11/14 21:00:00+00,NaN,NaN,25.9,NaN,NaN,75014 - Basch,Traffic
28965,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75014 - Basch,Traffic
28966,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75014 - Basch,Traffic


In [57]:
PA75014.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75014.csv", index=False)

In [58]:
quick_check2(PA75014,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, O3: 100.0%, SO2: 100.0%, NO2: 5.46%, PM10: 4.59%

You might want to drop these features: PM25, O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28968 entries, 0 to 28967
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     28968 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          27637 non-null  float64
 3   NO2           27387 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  28968 non-null  object 
 7   Station_type  28968 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.0+ MB
None


### 75015_U_Lenglen

In [59]:
file_list =  !find ../data/Pollution/75015_U_Lenglen | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75015 - Lenglen'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75015 - Lenglen'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75015_U_Lenglen.csv")

In [60]:
PA75015 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75015_U_Lenglen.csv", index_col=0).copy()
PA75015 = PA75015.rename(columns={
    "index":"Date_time",
    "PA15L:PM10":"PM10",
    "PA15L:NO2":"NO2",
    "PA15L:NO":"NO",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75015 = PA75015.drop(columns=['NO'])
PA75015['PM25'] = float('NaN')
PA75015['O3'] = float('NaN')
PA75015['SO2'] = float('NaN')
PA75015 = PA75015[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75015

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,14.7,4.0,NaN,NaN,75015 - Lenglen,Urbain
1,2018/01/01 02:00:00+00,NaN,14.1,4.0,NaN,NaN,75015 - Lenglen,Urbain
2,2018/01/01 03:00:00+00,NaN,11.2,4.0,NaN,NaN,75015 - Lenglen,Urbain
3,2018/01/01 04:00:00+00,NaN,12.8,4.0,NaN,NaN,75015 - Lenglen,Urbain
4,2018/01/01 05:00:00+00,NaN,16.2,4.0,NaN,NaN,75015 - Lenglen,Urbain
...,...,...,...,...,...,...,...,...
42139,2022/11/13 20:00:00+00,NaN,16.4,24.4,NaN,NaN,75015 - Lenglen,Urbain
42140,2022/11/13 21:00:00+00,NaN,22.0,27.1,NaN,NaN,75015 - Lenglen,Urbain
42141,2022/11/13 22:00:00+00,NaN,NaN,29.5,NaN,NaN,75015 - Lenglen,Urbain
42142,2022/11/13 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75015 - Lenglen,Urbain


In [61]:
PA75015.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75015.csv", index=False)

In [62]:
quick_check2(PA75015,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, O3: 100.0%, SO2: 100.0%, PM10: 18.35%, NO2: 4.46%

You might want to drop these features: PM25, O3, SO2, PM10


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42144 entries, 0 to 42143
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42144 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          34409 non-null  float64
 3   NO2           40264 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42144 non-null  object 
 7   Station_type  42144 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75016_T_Porte_Auteuil

In [63]:
file_list =  !find ../data/Pollution/75016_T_Porte_Auteuil | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75016 - Porte Auteuil'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75016 - Porte Auteuil'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75016_T_Porte_Auteuil.csv")

In [64]:
PA75016 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75016_T_Porte_Auteuil.csv", index_col=0).copy()
PA75016 = PA75016.rename(columns={
    "index":"Date_time",
    "AUT:CO":"CO",
    "AUT:PM10":"PM10",
    "AUT:PM25":"PM25",
    "AUT:NO2":"NO2",
    "AUT:SO2":"SO2",
    "AUT:ETBEN":"ETBEN",
    "AUT:m+pXYL":"m+pXYL",
    "AUT:NO":"NO",
    "AUT:oXYL":"oXYL",
    "AUT:NOX":"NOX",
    "AUT:TOL":"TOL",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75016 = PA75016.drop(columns=['CO', 'NO', 'NOX', 'ETBEN', 'm+pXYL', 'oXYL', 'TOL'])
PA75016['O3'] = float('NaN')
PA75016 = PA75016[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75016

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
1,2018/01/01 02:00:00+00,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
2,2018/01/01 03:00:00+00,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
3,2018/01/01 04:00:00+00,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
4,2018/01/01 05:00:00+00,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
...,...,...,...,...,...,...,...,...
42571,2022/11/14 20:00:00+00,14.5,18.8,41.5,NaN,NaN,75016 - Porte Auteuil,Traffic
42572,2022/11/14 21:00:00+00,NaN,NaN,34.5,NaN,NaN,75016 - Porte Auteuil,Traffic
42573,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic
42574,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75016 - Porte Auteuil,Traffic


In [65]:
PA75016.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75016.csv", index=False)

In [66]:
quick_check2(PA75016,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): O3: 100.0%, SO2: 40.24%, NO2: 4.27%, PM25: 3.11%, PM10: 2.7%

You might want to drop these features: O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42576 entries, 0 to 42575
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42576 non-null  object 
 1   PM25          41250 non-null  float64
 2   PM10          41426 non-null  float64
 3   NO2           40760 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           25444 non-null  float64
 6   Station_name  42576 non-null  object 
 7   Station_type  42576 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 75018_U_Flocon

In [67]:
file_list =  !find ../data/Pollution/75018_U_Flocon | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '75018 - Flocon'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '75018 - Flocon'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75018_U_Flocon.csv")

In [68]:
PA75018 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/75018_U_Flocon.csv", index_col=0).copy()
PA75018 = PA75018.rename(columns={
    "index":"Date_time",
    "PA18:PM10":"PM10",
    "PA18:NO2":"NO2",
    "PA18:NO":"NO",
    "PA18:NOX":"NOX",
    "PA18:O3":"O3",
    "PA18:PM25":"PM25",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA75018 = PA75018.iloc[:, [0, 8, 1, 2, 3, 4, 5, 6, 7]]
PA75018 = PA75018.drop(columns=['NO', 'NOX'])
PA75018['SO2'] = float('NaN')
PA75018 = PA75018[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA75018

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,14.2,14.0,66.0,NaN,75018 - Flocon,Urbain
1,2018/01/01 02:00:00+00,NaN,13.0,14.0,66.0,NaN,75018 - Flocon,Urbain
2,2018/01/01 03:00:00+00,NaN,11.6,14.0,64.0,NaN,75018 - Flocon,Urbain
3,2018/01/01 04:00:00+00,NaN,11.9,15.0,62.0,NaN,75018 - Flocon,Urbain
4,2018/01/01 05:00:00+00,NaN,14.5,14.0,62.0,NaN,75018 - Flocon,Urbain
...,...,...,...,...,...,...,...,...
42667,2022/11/13 20:00:00+00,20.5,23.2,28.3,0.7,NaN,75018 - Flocon,Urbain
42668,2022/11/13 21:00:00+00,22.6,22.5,27.9,1.2,NaN,75018 - Flocon,Urbain
42669,2022/11/13 22:00:00+00,NaN,NaN,NaN,NaN,NaN,75018 - Flocon,Urbain
42670,2022/11/13 23:00:00+00,NaN,NaN,NaN,NaN,NaN,75018 - Flocon,Urbain


In [69]:
PA75018.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA75018.csv", index=False)

In [70]:
quick_check2(PA75018,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): SO2: 100.0%, PM25: 83.1%, PM10: 3.62%, NO2: 3.31%, O3: 2.14%

You might want to drop these features: SO2, PM25


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42672 entries, 0 to 42671
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42672 non-null  object 
 1   PM25          7210 non-null   float64
 2   PM10          41127 non-null  float64
 3   NO2           41260 non-null  float64
 4   O3            41759 non-null  float64
 5   SO2           0 non-null      float64
 6   Station_name  42672 non-null  object 
 7   Station_type  42672 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 92220_U_Neuilly

In [71]:
file_list =  !find ../data/Pollution/92220_U_Neuilly | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '92220 - Neuilly'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '92220 - Neuilly'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/92220_U_Neuilly.csv")

In [72]:
PA92220 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/92220_U_Neuilly.csv", index_col=0).copy()
PA92220 = PA92220.rename(columns={
    "index":"Date_time",
    "NEUIL:NO2":"NO2",
    "NEUIL:SO2":"SO2",
    "NEUIL:NO":"NO",
    "NEUIL:NOX":"NOX",
    "NEUIL:O3":"O3",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA92220 = PA92220.drop(columns=['NO', 'NOX'])
PA92220['PM25'] = float('NaN')
PA92220['PM10'] = float('NaN')
PA92220 = PA92220[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA92220

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,11.0,74.0,1.0,92220 - Neuilly,Urbain
1,2018/01/01 02:00:00+00,NaN,NaN,11.0,75.0,1.0,92220 - Neuilly,Urbain
2,2018/01/01 03:00:00+00,NaN,NaN,13.0,71.0,1.0,92220 - Neuilly,Urbain
3,2018/01/01 04:00:00+00,NaN,NaN,15.0,66.0,1.0,92220 - Neuilly,Urbain
4,2018/01/01 05:00:00+00,NaN,NaN,12.0,68.0,1.0,92220 - Neuilly,Urbain
...,...,...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,NaN,NaN,27.0,20.4,NaN,92220 - Neuilly,Urbain
42692,2022/11/14 21:00:00+00,NaN,NaN,17.4,30.5,NaN,92220 - Neuilly,Urbain
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,92220 - Neuilly,Urbain
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,92220 - Neuilly,Urbain


In [73]:
PA92220.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA92220.csv", index=False)

In [74]:
quick_check2(PA92220,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, PM10: 100.0%, SO2: 37.7%, NO2: 4.34%, O3: 1.57%

You might want to drop these features: PM25, PM10, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          0 non-null      float64
 3   NO2           40844 non-null  float64
 4   O3            42027 non-null  float64
 5   SO2           26599 non-null  float64
 6   Station_name  42696 non-null  object 
 7   Station_type  42696 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 92800_U_La_Defense

In [75]:
file_list =  !find ../data/Pollution/92800_U_La_Defense | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '92800 - La Defense'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '92800 - La Defense'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/92800_U_La_Defense.csv")

In [76]:
PA92800 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/92800_U_La_Defense.csv", index_col=0).copy()
PA92800 = PA92800.rename(columns={
    "index":"Date_time",
    "DEF:PM10":"PM10",
    "DEF:PM25":"PM25",
    "DEF:NO2":"NO2",
    "DEF:NO":"NO",
    "DEF:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA92800 = PA92800.iloc[:, [0, 7, 1, 2, 3, 4, 5, 6]]
PA92800 = PA92800.drop(columns=['NO', 'NOX'])
PA92800['O3'] = float('NaN')
PA92800['SO2'] = float('NaN')
PA92800 = PA92800[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA92800

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,12.6,5.0,NaN,NaN,92800 - La Defense,Urbain
1,2018/01/01 02:00:00+00,NaN,10.1,5.0,NaN,NaN,92800 - La Defense,Urbain
2,2018/01/01 03:00:00+00,NaN,11.3,5.0,NaN,NaN,92800 - La Defense,Urbain
3,2018/01/01 04:00:00+00,NaN,10.2,7.0,NaN,NaN,92800 - La Defense,Urbain
4,2018/01/01 05:00:00+00,NaN,11.6,6.0,NaN,NaN,92800 - La Defense,Urbain
...,...,...,...,...,...,...,...,...
32923,2022/10/10 20:00:00+00,NaN,NaN,NaN,NaN,NaN,92800 - La Defense,Urbain
32924,2022/10/10 21:00:00+00,NaN,NaN,NaN,NaN,NaN,92800 - La Defense,Urbain
32925,2022/10/10 22:00:00+00,NaN,NaN,NaN,NaN,NaN,92800 - La Defense,Urbain
32926,2022/10/10 23:00:00+00,NaN,NaN,NaN,NaN,NaN,92800 - La Defense,Urbain


In [77]:
PA92800.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA92800.csv", index=False)

In [78]:
quick_check2(PA92800,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): O3: 100.0%, SO2: 100.0%, PM25: 96.45%, PM10: 45.26%, NO2: 26.0%

You might want to drop these features: O3, SO2, PM25, PM10, NO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32928 entries, 0 to 32927
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     32928 non-null  object 
 1   PM25          1170 non-null   float64
 2   PM10          18024 non-null  float64
 3   NO2           24366 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  32928 non-null  object 
 7   Station_type  32928 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.3+ MB
None


### 93300_U_Aubervilliers

In [79]:
file_list =  !find ../data/Pollution/93300_U_Aubervilliers | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '93300 - Aubervilliers'
base['Station_type'] = 'Urbain'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '93300 - Aubervilliers'
    new_df['Station_type'] = 'Urbain'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/93300_U_Aubervilliers.csv")

In [80]:
PA93300 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/93300_U_Aubervilliers.csv", index_col=0).copy()
PA93300 = PA93300.rename(columns={
    "index":"Date_time",
    "AUB:CO":"CO",
    "AUB:NO2":"NO2",
    "AUB:SO2":"SO2",
    "AUB:NO":"NO",
    "AUB:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA93300 = PA93300.drop(columns=['CO', 'NO', 'NOX'])
PA93300['PM25'] = float('NaN')
PA93300['PM10'] = float('NaN')
PA93300['O3'] = float('NaN')
PA93300 = PA93300[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA93300

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,NaN,12.0,NaN,0.0,93300 - Aubervilliers,Urbain
1,2018/01/01 02:00:00+00,NaN,NaN,13.0,NaN,0.0,93300 - Aubervilliers,Urbain
2,2018/01/01 03:00:00+00,NaN,NaN,12.0,NaN,0.0,93300 - Aubervilliers,Urbain
3,2018/01/01 04:00:00+00,NaN,NaN,14.0,NaN,0.0,93300 - Aubervilliers,Urbain
4,2018/01/01 05:00:00+00,NaN,NaN,13.0,NaN,0.0,93300 - Aubervilliers,Urbain
...,...,...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,NaN,NaN,40.1,NaN,NaN,93300 - Aubervilliers,Urbain
42692,2022/11/14 21:00:00+00,NaN,NaN,28.6,NaN,NaN,93300 - Aubervilliers,Urbain
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,93300 - Aubervilliers,Urbain
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,93300 - Aubervilliers,Urbain


In [81]:
PA93300.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA93300.csv", index=False)

In [82]:
quick_check2(PA93300,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, PM10: 100.0%, O3: 100.0%, SO2: 40.26%, NO2: 3.67%

You might want to drop these features: PM25, PM10, O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          0 non-null      float64
 3   NO2           41130 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           25506 non-null  float64
 6   Station_name  42696 non-null  object 
 7   Station_type  42696 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None


### 93500_T_RN2_Pantin

In [83]:
file_list =  !find ../data/Pollution/93500_T_RN2_Pantin | grep .csv
file_list = file_list.sort()

base = pd.read_csv(file_list[0], index_col=0).copy()
base = base.drop(columns="OBJECTID")
base['Station_name'] = '93500 - RN2 Pantin'
base['Station_type'] = 'Traffic'
base = base[5:]

for i in file_list[1:] :
    new_df = pd.read_csv(i,index_col=0).copy()
    new_df = new_df.drop(columns="OBJECTID")
    new_df['Station_name'] = '93500 - RN2 Pantin'
    new_df['Station_type'] = 'Traffic'
    new_df = new_df[5:]
    base = pd.concat([base, new_df])

base = base.reset_index()
    
base.to_csv(f"{LOCAL_DATA_PATH_pollution_merged}/93500_T_RN2_Pantin.csv")

In [84]:
PA93500 = pd.read_csv(f"{LOCAL_DATA_PATH_pollution_merged}/93500_T_RN2_Pantin.csv", index_col=0).copy()
PA93500 = PA93500.rename(columns={
    "index":"Date_time",
    "RN2:PM10":"PM10",
    "RN2:NO2":"NO2",
    "RN2:NO":"NO",
    "RN2:NOX":"NOX",
    "Station_name":"Station_name",
    "Station_type":"Station_type"}, errors="raise")
PA93500 = PA93500.drop(columns=['NO', 'NOX'])
PA93500['PM25'] = float('NaN')
PA93500['O3'] = float('NaN')
PA93500['SO2'] = float('NaN')
PA93500 = PA93500[['Date_time', 'PM25', 'PM10', 'NO2', 'O3', 'SO2', 'Station_name', 'Station_type']]
PA93500

,Date_time,PM25,PM10,NO2,O3,SO2,Station_name,Station_type
0,2018/01/01 01:00:00+00,NaN,16.6,18.0,NaN,NaN,93500 - RN2 Pantin,Traffic
1,2018/01/01 02:00:00+00,NaN,8.5,19.0,NaN,NaN,93500 - RN2 Pantin,Traffic
2,2018/01/01 03:00:00+00,NaN,6.6,21.0,NaN,NaN,93500 - RN2 Pantin,Traffic
3,2018/01/01 04:00:00+00,NaN,17.4,22.0,NaN,NaN,93500 - RN2 Pantin,Traffic
4,2018/01/01 05:00:00+00,NaN,18.4,19.0,NaN,NaN,93500 - RN2 Pantin,Traffic
...,...,...,...,...,...,...,...,...
42691,2022/11/14 20:00:00+00,NaN,27.4,48.6,NaN,NaN,93500 - RN2 Pantin,Traffic
42692,2022/11/14 21:00:00+00,NaN,NaN,36.8,NaN,NaN,93500 - RN2 Pantin,Traffic
42693,2022/11/14 22:00:00+00,NaN,NaN,NaN,NaN,NaN,93500 - RN2 Pantin,Traffic
42694,2022/11/14 23:00:00+00,NaN,NaN,NaN,NaN,NaN,93500 - RN2 Pantin,Traffic


In [85]:
PA93500.to_csv(f"{LOCAL_DATA_PATH_pollution_processed}/PA93500.csv", index=False)

In [86]:
quick_check2(PA93500,"NO2")

You have : 5 features over 8 (0.62% of whole df) that include np.nan

Have a look at these features (% of nan): PM25: 100.0%, O3: 100.0%, SO2: 100.0%, PM10: 2.35%, NO2: 2.04%

You might want to drop these features: PM25, O3, SO2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42696 entries, 0 to 42695
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date_time     42696 non-null  object 
 1   PM25          0 non-null      float64
 2   PM10          41694 non-null  float64
 3   NO2           41825 non-null  float64
 4   O3            0 non-null      float64
 5   SO2           0 non-null      float64
 6   Station_name  42696 non-null  object 
 7   Station_type  42696 non-null  object 
dtypes: float64(5), object(3)
memory usage: 2.9+ MB
None
